<a href="https://colab.research.google.com/github/izactxe/HelloGit/blob/master/kaggle_titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

# load data

In [0]:
train_url = "http://s3.amazonaws.com/assets.datacamp.com/course/Kaggle/train.csv"
train = pd.read_csv(train_url) #training set

test_url = "http://s3.amazonaws.com/assets.datacamp.com/course/Kaggle/test.csv"
test = pd.read_csv(test_url) #test set

In [0]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [0]:
train.tail()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.00,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


In [0]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [0]:
train.notnull().mean()  # fill rate

PassengerId    1.000000
Survived       1.000000
Pclass         1.000000
Name           1.000000
Sex            1.000000
Age            0.801347
SibSp          1.000000
Parch          1.000000
Ticket         1.000000
Fare           1.000000
Cabin          0.228956
Embarked       0.997755
dtype: float64

# T7

In [0]:
train.Age.median()

28.0

In [0]:
train["Age"] = train["Age"].fillna(train["Age"].median())
test["Age"] = test["Age"].fillna(train["Age"].median())

In [0]:
train.notnull().mean()  # fill rate

PassengerId    1.000000
Survived       1.000000
Pclass         1.000000
Name           1.000000
Sex            1.000000
Age            1.000000
SibSp          1.000000
Parch          1.000000
Ticket         1.000000
Fare           1.000000
Cabin          0.228956
Embarked       0.997755
dtype: float64

# T8

In [0]:
train.Embarked.unique()

array(['S', 'C', 'Q', nan], dtype=object)

In [0]:
train['Embarked'] = train.Embarked.map({'S': 0, 'C': 1, 'Q': 2})
test['Embarked'] = test.Embarked.map({'S': 0, 'C': 1, 'Q': 2})

In [0]:
train['Embarked'].fillna(train['Embarked'].mode()[0], inplace=True)
test['Embarked'].fillna(train['Embarked'].mode()[0], inplace=True)

In [0]:
train.Sex.unique()

array(['male', 'female'], dtype=object)

In [0]:
train['Sex'] = train.Sex.map({'male': 0, 'female': 1})
test['Sex'] = test.Sex.map({'male': 0, 'female': 1})

In [0]:
train.describe()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,0.352413,29.361582,0.523008,0.381594,32.204208,0.361392
std,257.353842,0.486592,0.836071,0.477990,13.019697,1.102743,0.806057,49.693429,0.635673
min,1.000000,0.000000,1.000000,0.000000,0.420000,0.000000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,0.000000,22.000000,0.000000,0.000000,7.910400,0.000000
50%,446.000000,0.000000,3.000000,0.000000,28.000000,0.000000,0.000000,14.454200,0.000000
75%,668.500000,1.000000,3.000000,1.000000,35.000000,1.000000,0.000000,31.000000,1.000000
max,891.000000,1.000000,3.000000,1.000000,80.000000,8.000000,6.000000,512.329200,2.000000


# T9

In [0]:
class MyLogReg:
    def __init__(self, lr=1e-5, n_no_impv=1e4):
        self.theta = None
        self.lr = lr
        self.n_no_impv = n_no_impv
        self.random_seed = 1234
    
    def fit(self, X, y):
        if type(X) is not np.ndarray:
            X = np.array(X.values)
        y = np.array(y)
        
        np.random.seed(self.random_seed)
        self.theta = np.random.randn(X.shape[1])
        
        count_no_impv = 0
        best_acc = 0
        best_theta = None
        while True:
            prob = self.predict_prob(X)
            self.theta = self.theta + self.lr * X.T.dot(y - prob)
            
            acc = (self.predict(X) == y.astype(bool)).mean()
            if acc < best_acc:
                count_no_impv += 1
            else:
                best_acc = acc
                best_theta = self.theta
                count_no_impv = 0

            if count_no_impv >= self.n_no_impv:
                break
        self.theta = best_theta

    def predict_prob(self, X):
        if type(X) is not np.ndarray:
            X = np.array(X.values)
        score = X.dot(self.theta)
        return 1 / (1 + np.exp(-score))

    def predict(self, X):
        prob = self.predict_prob(X)
        return prob >= 0.5

In [0]:
features = ['Pclass', 'Sex', 'Age', 'Embarked']
cls = MyLogReg()
cls.fit(train[features], train.Survived)
print('weight:', cls.theta)

weight: [-0.54783675  1.66272719 -0.00320822  0.34674829]


In [0]:
(cls.predict(train[features]) == train.Survived.astype(bool)).mean()

0.8125701459034792

In [0]:
(pd.concat([test.PassengerId, pd.Series(cls.predict(test[features])).rename('Survived').astype(int)], axis=1)
 .head())

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


In [0]:
(pd.concat([test.PassengerId, pd.Series(cls.predict(test[features])).rename('Survived').astype(int)], axis=1)
 .to_csv('test_result2.csv', index=False))

In [0]:
# test score: 0.77033
# result stored to T9.png

# OT4

Training accuracy increases. And for some reason, testing accuracy magically increases (slightly), even if I added many intercorrelated features and changed random state.

In [0]:
train['Pclass_2'] = train.Pclass ** 2
train['Age_2'] = train.Age ** 2 / 50
train['Age_3'] = train.Age ** 3 / 10000
train['Age_Sex'] = train.Age * train.Sex

test['Pclass_2'] = test.Pclass ** 2
test['Age_2'] = test.Age ** 2 / 50
test['Age_3'] = test.Age ** 3 / 10000
test['Age_Sex'] = test.Age * test.Sex

In [0]:
features = ['Pclass', 'Sex', 'Age', 'Embarked', 'Age_2', 'Age_3', 'Age_Sex', 'Pclass_2']
cls = MyLogReg()
cls.fit(train[features], train.Survived)
print('weight:', cls.theta)

weight: [ 1.75086994  1.08218253 -0.08857936  0.46841518  0.02653062 -0.03969774
  0.05542903 -0.69673453]


In [0]:
(cls.predict(train[features]) == train.Survived.astype(bool)).mean()

0.8237934904601572

In [0]:
(pd.concat([test.PassengerId, pd.Series(cls.predict(test[features])).rename('Survived').astype(int)], axis=1)
 .to_csv('OT4.csv', index=False))

In [0]:
# test score: 0.78947
# result stored to OT4.png

# OT5

Training accuracy drops to 0.79, as well as testing accuracy drops to 0.76.

In [0]:
features = ['Sex', 'Age']
cls = MyLogReg()
cls.fit(train[features], train.Survived)
print('weight:', cls.theta)

weight: [ 0.50134724 -0.00706975]


In [0]:
(cls.predict(train[features]) == train.Survived.astype(bool)).mean()

0.7867564534231201

In [0]:
(pd.concat([test.PassengerId, pd.Series(cls.predict(test[features])).rename('Survived').astype(int)], axis=1)
 .to_csv('OT5.csv', index=False))

In [0]:
# test score: 0.76076
# result stored to OT5.png